In [1]:
from torchvision import transforms
from helper import get_preprocessed_data
train_path = "/workspace/Comp/Datasets/Project/training/images"
groundtruth_path = "/workspace/Comp/Datasets/Project/training/groundtruth"

## Scale the image between 0 and 1 and resize it to (416, 416)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((416, 416)),
    ])


X, Y = get_preprocessed_data(train_path, transform, groundtruth_path)
print(X.shape, Y.shape)

torch.Size([144, 3, 416, 416]) torch.Size([144, 1, 416, 416])


In [2]:
import torch
import torch.nn as nn

class StackedModel(nn.Module):
    def _init_(self, pspnet, deeplab, combine_method='average'):
        super(StackedModel, self)._init_()
        self.pspnet = pspnet
        self.deeplab = deeplab
        self.combine_method = combine_method

    def forward(self, x):
        pspnet_output = self.pspnet(x)
        deeplab_output = self.deeplab(x)

        if self.combine_method == 'average':
            combined_output = (pspnet_output + deeplab_output) / 2
        elif self.combine_method == 'weighted':
            alpha = 0.6 
            combined_output = alpha * pspnet_output + (1 - alpha) * deeplab_output
        elif self.combine_method == 'max':
            combined_output = torch.max(pspnet_output, deeplab_output)
        else:
            raise ValueError("Unknown combination method")

        return combined_output

In [3]:
import torch

deeplabv3resnet152 = torch.load('/workspace/Comp/Models/deeplabv3resnet152.pth')
advanced_resnet50 = torch.load('/workspace/Comp/Models/eliotu_advanced_resnet50.pth')
pspnet = torch.load('/workspace/Comp/Models/ryounis_pspnet_deeplabv3.pth')
deeplabv3resnet101 = torch.load('/workspace/Comp/Models/deeplabv3resnet101.pth')

models = [deeplabv3resnet152, advanced_resnet50, pspnet, deeplabv3resnet101]

/tmp/ipykernel_5874/3934499941.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  deeplabv3resnet152 = torch.load('/workspace/Comp/Models/deeplabv3resnet152.pth')
/workspac

In [8]:
import optuna
from helper import get_mul_f1_score

def objective(trial):
    model_weights = [trial.suggest_float(f'model_{i}', 0.15, 0.85) for i in range(len(models))]
    threshold = trial.suggest_float('threshold', 0.3, 0.7)
    
    
    return -get_mul_f1_score(models, model_weights, zip(X, Y), threshold)

# study = optuna.create_study(storage="sqlite:///t2.db", study_name="t2")
study = optuna.load_study(storage="sqlite:///optf1-4-models.db", study_name="optf1-4-models")

# study.optimize(objective, n_trials=30)

print(study.best_params)  # E.g. {'x': 2.002108042}
print(study.best_value)  # E.g. 3.0


{'model_0': 0.6287562002237642, 'model_1': 0.39893119998558085, 'model_2': 0.23367054410526328, 'model_3': 0.8383812067643728, 'threshold': 0.5931206715720474}
-0.8579478225461616


In [10]:
from mask_to_submission import *
from PIL import Image
import numpy as np
import os
import torch

test_path = "/workspace/Comp/Datasets/Project/test/images"

X_test, _ = get_preprocessed_data('/workspace/Comp/Datasets/Project/test/images', transform, None)
X_test_files = sorted(os.listdir(test_path))

best_model_weights = [study.best_params[f'model_{i}'] for i in range(len(models))]
best_threshold = study.best_params['threshold']

all_preds = []
for model in models:
    model.eval()

with torch.no_grad():  # Disable gradient computation
    for images in X_test:
        images = images.to('cuda')
        predictions = torch.sum(torch.stack([model(torch.stack([images])) * best_model_weights[i] for i, model in enumerate(models)]), 2)
        
        prediction = (predictions[0] > best_threshold).cpu().numpy()
        all_preds.append(prediction)
        
# Concatenate all predictions
y_pred = np.concatenate(all_preds, axis=0)

# Ensure the predictions are of shape (height, width) before converting to images
print(y_pred.shape)
y_pred = (y_pred).astype(np.uint8)  # Squeeze the last dimension if necessary

pred_images = [Image.fromarray((y_pred[i] * 255)).resize((400, 400)) for i in range(y_pred.shape[0])]

output_path = '/workspace/Comp/Road-Segmentation-Comp/outputs'
os.makedirs(output_path, exist_ok=True)  # Ensure the output directory exists

# Save all images in the output_path directory
for i, img in enumerate(pred_images):
    img.save(f'{output_path}/pred_{X_test_files[i]}')

print("Images have been successfully saved.")

(144, 416, 416)
Images have been successfully saved.
